# Controle de carteiras Exponential Coins

## Pacotes necessários

In [ ]:
#!pip install requests
#!pip install pandas
!pip install pycoingecko
#!pip install numpy
#!pip install plotly


## Autorização Google Sheets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import auth
import gspread
from google.auth import default

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

Mounted at /content/drive


## Classe Carteira

Este objeto python é uma classe que realiza uma série de funções para facilitar o controle de carteira de cripto ativos, requerendo apenas agluns argumentos:

1. Nome da Carteira
2. Lista de ativos (ids de API do CoinGecko)
3. Alocação (manuseadas via sheets e importadas nesse notebook)
4. Categorias (manuseadas via sheets e importadas nesse notebook)

O objeto cria um diretório de ativos em pastas individuais, e atualiza-as individualmente toda vez que o script é rodado.

O diretório de ativos é mapeado para um dataframe consolidado, com base na planilha de alocações. Portanto, todo o controle desse objeto está baseado na:


1. Atualização de planilhas (criação de colunas e atualização de alocações)
2. Atualização da lista de ids de API para cada carteira (cada uma tem sua lista).

In [ ]:
import os
import time
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from pycoingecko import CoinGeckoAPI
import warnings
import requests
import plotly.graph_objects as go

warnings.filterwarnings("ignore")


class PortfolioCrypto:

    cg = CoinGeckoAPI()
    _coins = None

    @classmethod
    def get_coins_list(cls):
        if cls._coins is None:
            cls._coins = cls.cg.get_coins_list()
        return cls._coins


    def __init__(self, carteira, ativos, aloc, categorias):
        self.nome = carteira
        self.ativos = ativos
        self.aloc = aloc
        self.categorias = categorias
        self.df_aloc = pd.DataFrame(self.aloc)
        self.df_cats = pd.DataFrame(self.categorias)
        self.df_aloc_balanceada_alerta = pd.DataFrame()
        self.df_aloc_balanceada_periodo = pd.DataFrame()
        self.df_precos = pd.DataFrame()
        self.df_ptax = pd.DataFrame()
        self.df_btc = pd.DataFrame()
        self.df_retornos = pd.DataFrame()
        self.df_retornos_ponderados = pd.DataFrame()
        self._atualizar_cotacoes()
        self.ptax()
        self._get_filtered_coins()
        self.precos()
        self.calcular_retornos()
        self.calcular_retornos_ponderados()
        self.calcular_posicao_balanceada_alerta()
        self.mapeamento_ticker_id = {}

    def _get_filtered_coins(self):
      coins_df = pd.DataFrame(self.get_coins_list())
      filtered_coins_df = coins_df[coins_df['id'].isin(self.ativos)]
      self.mapeamento_ticker_id = {row['symbol'].upper(): row['id'] for index, row in filtered_coins_df.iterrows()}


    def _atualizar_cotacoes(self):
      total_ativos = len(self.ativos)

      # Contador de ativos efetivamente processados
      ativos_processados = 0

      for i, ativo in enumerate(self.ativos, start=1):

          print(f"Processando {i} de {total_ativos}: {ativo}")

          arquivo = f"/ativos_exc/{ativo}.csv"

          if os.path.exists(arquivo):
              df_existente = pd.read_csv(arquivo, parse_dates=['data'], index_col='data')
              ultima_data = df_existente.index[-1]

              if ultima_data == (datetime.now() - timedelta(days=1)).date():
                print(f"{ativo} OK!")
                continue

              dias_para_atualizar = (datetime.now() - ultima_data).days - 1
          else:
              dias_para_atualizar = 10000

          try:
              dados = self.cg.get_coin_market_chart_by_id(id=ativo, vs_currency='usd', days=str(dias_para_atualizar), interval='daily')

              # Incremente o contador de ativos efetivamente processados
              ativos_processados += 1

          except Exception as e:
              print(f"Erro ao buscar dados para {ativo}: {e}")
              continue

          df_novo = pd.DataFrame(dados['prices'], columns=['data', 'close'])
          df_novo['data'] = pd.to_datetime(df_novo['data'], unit='ms').dt.normalize()

          # Remove a última linha (preço atual)
          df_novo = df_novo.iloc[:-1]

          # Ajusta a data para o fechamento do dia anterior
          df_novo['data'] = df_novo['data'] - timedelta(days=1)

          df_novo.set_index('data', inplace=True)

          # Certifique-se de que o índice seja único
          df_novo = df_novo[~df_novo.index.duplicated(keep='first')]

          # Interpolar dados se houver gaps
          df_novo = df_novo.asfreq('D', method='pad')

          if os.path.exists(arquivo):
              df_atualizado = pd.concat([df_existente, df_novo]).drop_duplicates()
          else:
              df_atualizado = df_novo

          df_atualizado.to_csv(arquivo)

          # Verificar pausa da API usando o contador de ativos efetivamente processados
          if ativos_processados % 15 == 0:
              print(f"API cooldown (1m)...")
              time.sleep(61)



    def precos(self):

        self._get_filtered_coins()
        self.df_aloc.columns = map(str.upper, self.df_aloc.columns)
        df_precos = pd.DataFrame(index=self.df_aloc.index)

        for coluna in self.df_aloc.columns:
            if coluna in self.mapeamento_ticker_id:
                id_ativo = self.mapeamento_ticker_id[coluna]

                try:
                    df_ativo = pd.read_csv(f"/ativos_exc/{id_ativo}.csv", index_col='data', parse_dates=True)
                    df_ativo = df_ativo.groupby(df_ativo.index).first()

                    # Caso 1: Ativo já existia antes da carteira ser criada

                    if df_ativo.index[0] <= self.df_aloc.index[0]:
                        df_precos[coluna] = df_ativo['close'].reindex(self.df_aloc.index, method='ffill')

                    # Caso 2: Ativo foi criado depois do lançamento da carteira
                    else:
                        first_price = df_ativo['close'].iloc[0]  # Primeiro preço disponível
                        df_precos[coluna] = np.nan  # Inicialmente, preenche com NaN
                        df_precos.loc[df_ativo.index[0]:, coluna] = df_ativo['close']  # Preenche com preços reais quando disponíveis
                        df_precos.loc[:df_ativo.index[0], coluna] = first_price  # Preenche com o primeiro preço até a data de início

                except FileNotFoundError:
                    print(f"Arquivo para {id_ativo} não encontrado.")



        self.df_precos = df_precos



        return self.df_precos


    def calcular_retornos(self):

        df_retornos = self.df_precos.pct_change()
        df_retornos.iloc[0, :] = 0

        stablecoins = ['TUSD', 'BUSD', 'USDT']
        for coin in stablecoins:
            if coin in df_retornos.columns:
                df_retornos[coin] = 0


        self.df_retornos = df_retornos
        return self.df_retornos


    def calcular_posicao_balanceada_alerta(self):

        cot_init = 10000

        df_aloc = self.df_aloc
        df_retornos = self.df_retornos

        df_aloc = df_aloc.applymap(lambda x: float(x.strip('%')) / 100 if isinstance(x, str) and '%' in x else x)

        df_posicao = pd.DataFrame(index=df_aloc.index, columns=df_aloc.columns)

        df_posicao.iloc[0] = (df_aloc.iloc[0]) * cot_init

        for i in range(1,len(df_posicao)):

          df_posicao.iloc[i] = df_posicao.iloc[i-1] * (1 + df_retornos.iloc[i])

          if not df_aloc.iloc[i].equals(df_aloc.iloc[i-1]):

            valor_portfolio = df_posicao.iloc[i-1].sum()
            df_posicao.iloc[i] = (df_aloc.iloc[i]) * valor_portfolio
            df_posicao.iloc[i] = df_posicao.iloc[i] * (1 + df_retornos.iloc[i])

        self.df_aloc_balanceada_alerta = df_posicao.div(df_posicao.sum(axis=1), axis=0)

        df_posicao[self.nome] = df_posicao.sum(axis=1)

        self.df_posicao_balanceada_alerta = df_posicao

        return self.df_posicao_balanceada_alerta

    def calcular_retornos_ponderados(self):

        df_retornos = self.df_retornos
        df_aloc = self.df_aloc

        # planilhas do google as alocs vem como strings, por isso se converte (abaixo)
        df_aloc = df_aloc.applymap(lambda x: float(x.strip('%')) / 100 if isinstance(x, str) and '%' in x else x)

        df_retornos_ponderados = df_retornos.mul(df_aloc)

        df_retornos_ponderados[self.nome] = df_retornos_ponderados.sum(axis=1)

        self.df_retornos_ponderados = df_retornos_ponderados

        return self.df_retornos_ponderados


    def calcular_retornos_acumulados(self, inicio, fim, ponderar=False):

        # Retornos Acumulados Não Ponderados
        df_retornos = self.df_retornos.loc[inicio:fim]
        df_retornos.iloc[0, :] = 0  # Preencher a primeira linha com 0%


        if 'BTC' in df_retornos.columns:
          self.df_btc = df_retornos[['BTC']]


        self.df_retornos_acumulados = (1 + df_retornos).cumprod() - 1


        # Retornos Ponderados Acumulados
        df_retornos_ponderados = self.df_retornos_ponderados.loc[inicio:fim]
        df_retornos_ponderados.iloc[0, :] = 0  # Preencher a primeira linha com 0%
        self.df_retornos_acumulados_ponderados = (1+df_retornos_ponderados).cumprod() - 1


        # Remover colunas onde todos os elementos são iguais a zero
        self.df_retornos_acumulados = self.df_retornos_acumulados.loc[:, ~(self.df_retornos_acumulados == 0).all(axis=0)]
        self.df_retornos_acumulados_ponderados = self.df_retornos_acumulados_ponderados.loc[:, ~(self.df_retornos_acumulados_ponderados == 0).all(axis=0)]

        # Retorno Acumulado da Carteira

        self.df_retornos_acumulados[self.nome] = self.df_retornos_acumulados_ponderados[[self.nome]]

        if ponderar:
            return self.df_retornos_acumulados_ponderados
        else:
            return self.df_retornos_acumulados

    def ptax(self):
        start_date = "01/01/2017"
        end_date = datetime.today().strftime('%d/%m/%Y')

        base_url = "https://api.bcb.gov.br/dados/serie/bcdata.sgs.10813/dados"
        url = f"{base_url}?formato=json&dataInicial={start_date}&dataFinal={end_date}"

        response = requests.get(url)
        response.raise_for_status()

        data = response.json()
        df = pd.DataFrame(data)
        df['valor'] = df['valor'].str.replace(',', '.').astype(float)
        df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')
        df.set_index('data', inplace=True)
        df = df.resample('D').ffill()
        df['ptax'] = df['valor'].pct_change().fillna(0)
        df = df[['ptax']]

        self.df_ptax = df

        return self.df_ptax


    def calcular_var_historico(self, ativo, nivel_confianca=95, janela_amostral=365, periodo=7):
        """
        Calcula o Value at Risk (VaR) histórico para um ativo específico.

        :param ativo: String, o nome do ativo na carteira.
        :param nivel_confianca: Float, o nível de confiança para o cálculo do VaR (padrão 95%).
        :param janela_dias: Int, o número de dias para a amostragem histórica (padrão 365 dias).
        :return: Float, o VaR do ativo.
        """

        if ativo not in self.df_retornos.columns:
            raise ValueError(f"O ativo {ativo} não está presente na carteira.")

        # Selecionar os retornos do ativo para o período especificado
        retornos_recentes = self.df_retornos[ativo].tail(janela_amostral)
        retornos_acumulados_periodo = (1 + retornos_recentes).rolling(window=periodo).apply(np.prod, raw=True) - 1


        # Calcular o VaR como o percentil da distribuição de retornos
        var = np.percentile(retornos_acumulados_periodo.dropna(), 100 - nivel_confianca).round(4) * 100

        return var

    def calcular_voo_historico(self, ativo, nivel_confianca=95, janela_amostral=365, periodo=7):
        """
        Calcula o Value at Opportunity (VoO) histórico para um ativo específico.

        :param ativo: String, o nome do ativo na carteira.
        :param nivel_confianca: Float, o nível de confiança para o cálculo do VoO (padrão 95%).
        :param janela_dias: Int, o número de dias para a amostragem histórica (padrão 365 dias).
        :param periodo: Int, o número de dias para acumular os retornos (padrão 1, para retornos diários).
        :return: Float, o VoO do ativo.
        """
        if ativo not in self.df_retornos.columns:
            raise ValueError(f"O ativo {ativo} não está presente na carteira.")

        # Selecionar os retornos do ativo para o período especificado
        retornos_recentes = self.df_retornos[ativo].tail(janela_amostral)

        # Se o período é maior que 1 dia, calcular os retornos acumulados
        if periodo > 1:
            retornos_recentes = (1 + retornos_recentes).rolling(window=periodo).apply(np.prod, raw=True) - 1

        # Calcular o VoO como o percentil superior da distribuição de retornos
        voo = np.percentile(retornos_recentes.dropna(), nivel_confianca).round(4) * 100

        return voo

    def beta(self, inicio, fim):

        # Consertar

        df_carteira = self.df_retornos_ponderados[[self.nome]]
        df_ret = self.df_retornos[inicio:fim]
        df_ret = df_ret.merge(df_carteira[[self.nome]], on='Data')


        df_btc = pd.read_csv('/ativos_exc/bitcoin.csv', index_col='data').pct_change().fillna(0)
        df_btc = df_btc[inicio:fim]
        df_btc.index.rename('Data', inplace=True)
        df_btc.rename(columns={'close': 'BTC'}, inplace=True)
        df_btc.index = pd.to_datetime(df_btc.index)


        if 'BTC' not in df_ret.columns:
          df_ret = df_ret.merge(df_btc[['BTC']], on='Data')



        btc_var = df_ret['BTC'].var()

        betas = {}

        for ativo in df_ret.columns:
          if ativo != 'BTC':
            covariance = df_ret[ativo].cov(df_ret['BTC'])

            beta = round(covariance / btc_var, 4)

            betas[ativo] = beta

        return betas

    def rolling_beta(self, inicio, fim, janela=30):

        inicio = pd.to_datetime(inicio)
        fim = pd.to_datetime(fim)

        rolling_betas = pd.DataFrame()


        for start in pd.date_range(inicio, fim - pd.Timedelta(days=janela)):

            end = start + pd.Timedelta(days=janela)


            betas = self.beta(start.strftime('%Y-%m-%d'), end.strftime('%Y-%m-%d'))


            betas_df = pd.DataFrame(betas, index=[end])

            rolling_betas = pd.concat([rolling_betas, betas_df])

        return rolling_betas

    def posicoes_fechadas(self):

        df_aloc = self.df_aloc.applymap(lambda x: float(x.strip('%')) / 100 if isinstance(x, str) and '%' in x else x)
        df_preco = self.df_precos
        resultados = []

        stablecoins = ['USDT', 'BUSD', 'TUSD']

        for ativo in df_aloc.columns:
          if ativo not in stablecoins:
            alocacao_ativo = df_aloc[ativo]
            preco_ativo = df_preco[ativo]

            dentro_da_carteira = False

            for i in range(len(alocacao_ativo)):
                if alocacao_ativo[i] != 0 and not dentro_da_carteira:
                    # Início do período
                    data_entrada = alocacao_ativo.index[i]
                    preco_entrada = round(preco_ativo[data_entrada],2)
                    dentro_da_carteira = True

                elif alocacao_ativo[i] == 0 and dentro_da_carteira:
                    # Fim do período
                    data_saida = alocacao_ativo.index[i]
                    preco_saida = round(preco_ativo[data_saida],2)
                    retorno = round(((preco_saida - preco_entrada) / preco_entrada) * 100, 2)
                    resultados.append([ativo, data_entrada, data_saida, preco_entrada, preco_saida, retorno])
                    dentro_da_carteira = False

        results = pd.DataFrame(resultados, columns=['ativo', 'data_entrada', 'data_saida', 'preco_entrada', 'preco_saida', 'retorno_acumulado (%)']).sort_values(by=['retorno_acumulado (%)'], ascending=False, ignore_index=True)
        results.to_csv(f'historico_de_recomendaçoes_{self.nome}.csv')
        return results

    import pandas as pd

    def posicoes_abertas(self):
        df_aloc = self.df_aloc.applymap(lambda x: float(x.strip('%')) / 100 if isinstance(x, str) and '%' in x else x)
        df_preco = self.df_precos
        resultados = []

        stablecoins = ['USDT', 'BUSD', 'TUSD']
        ultima_linha = df_aloc.iloc[-1]  # Última linha da planilha de alocações

        for ativo in ultima_linha.index:
            if ativo not in stablecoins and ultima_linha[ativo] != 0:
                alocacao_ativo = df_aloc[ativo]
                preco_ativo = df_preco[ativo]

                # Encontrar a última data de entrada
                datas_entrada = alocacao_ativo[alocacao_ativo.ne(0) & alocacao_ativo.shift(1).eq(0)].index
                if len(datas_entrada) > 0:
                    ultima_entrada = datas_entrada[-1]
                elif alocacao_ativo.ne(0).all():
                    # Se o ativo sempre esteve na carteira, usar a primeira data do DataFrame
                    ultima_entrada = alocacao_ativo.index[0]

                preco_entrada = round(preco_ativo[ultima_entrada], 2)
                preco_atual = round(preco_ativo.iloc[-1], 2)
                valorizacao = round(((preco_atual - preco_entrada) / preco_entrada) * 100, 2) if preco_entrada != 0 else 0

                resultados.append([ativo, ultima_entrada, preco_entrada, preco_atual, valorizacao])



        results = pd.DataFrame(resultados, columns=['ativo', 'ultima_entrada', 'preco_entrada', 'preco_atual', 'retorno_acumulado(%)']).sort_values(by=['retorno_acumulado(%)'], ascending=False ,ignore_index=True)
        results.to_csv(f'posicoes_abertas_{self.nome}.csv')
        return results






## Funções auxiliares

As funções abaixo recebem uma instância de carteira, e possivelmente mais alguns outros argumentos, como datas de `início` e `fim`.

### Plotar resultado das carteiras

Argumentos da função:
1. `inicio`: 'YYYY-MM-DD'
2. `fim`: 'YYYY-MM-DD'
3. `carteira`: *exc*, *hb* ou *lc*
4. `brl`: True ou False (retorna a valorização em BRL)

In [ ]:
def plot_carteiras(inicio, fim, *carteiras, brl=False):

    # Calcula os retornos acumulados ponderados para cada carteira dentro das datas especificadas
    data_frames = [carteira.calcular_retornos_acumulados(inicio, fim, ponderar=True)[carteira.nome] for carteira in carteiras]

    # Adiciona o retorno acumulado do BTC
    btc = carteiras[0].df_btc.loc[inicio:fim]
    btc.iloc[0, :] = 0
    btc_acumulado = (1 + btc).cumprod() - 1
    btc_acumulado = btc_acumulado.rename(columns={"btc": "BTC"})
    data_frames.append(btc_acumulado)
    df = pd.concat(data_frames, axis=1)

    if brl:
      ptax = carteiras[0].df_ptax.loc[inicio:fim]
      ptax.iloc[0, :] = 0
      ptax['ptax_acumulado'] = (1+ptax).cumprod() - 1
      ptax = ptax[['ptax_acumulado']]
      df = (1+df).multiply(1+ptax['ptax_acumulado'], axis = 0) - 1



    df = df * 100



    # Plota o gráfico usando Plotly
    fig = go.Figure()

    for col in df.columns:
        fig.add_trace(go.Scatter(x=df.index, y=df[col], mode='lines', name=col,
                                 hovertemplate='%{y:.2f}%',
                                 line_shape='spline',  # Makes the lines smoother
                                 line=dict(width=2)))  # Adjust line width


    titulo = 'Rentabilidade Carteiras Exponential Coins (BRL)' if brl else 'Rentabilidade Carteiras Exponential Coins (USD)'

    # Update layout to make it cleaner
    fig.update_layout(

        margin=dict(t=60),

        title={'text': titulo,
               'x': 0.055,  # Centering the title
               'y': 0.97,  # Position adjustment to leave space for subtitle
               'font': {'size': 18,  # Larger font size for title
                        'color': 'black',
                        'family': 'Georgia'}},  # Elegant serif font for title
        xaxis_title='',
        yaxis_title='Rentabilidade Acumulada (%)',
        legend_title="Carteiras e benchmarks: ",
        hovermode='x unified',
        plot_bgcolor='white',  # White background
        xaxis=dict(
            showline=True,
            showgrid=True,
            gridcolor='lightgray',  # Light gray grid lines
            linecolor='white',  # Remove border
            linewidth=2,
            zerolinecolor='lightgray'
        ),
        yaxis=dict(
            showline=True,
            showgrid=True,
            gridcolor='lightgray',
            linecolor='white',
            linewidth=2,
            zerolinecolor='lightgray'
        ),
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=1.02,
            xanchor='right',
            x=1
        ),
        font=dict(
            family='Georgia',  # Update font family (optional)
            size=15,  # Adjust font size
            color='black'  # Font color
        ),
        annotations=[  # Adding the subtitle
            dict(
                text=f'Período: {inicio} à {fim}',  # Subtitle text
                xref='paper', yref='paper',  # Reference to paper (entire figure)
                x=0.00001, y=1.065,  # Position of subtitle (underneath title)
                showarrow=False,  # We don't need an arrow pointing to our subtitle
                font=dict(
                    family='Georgia',
                    size=13,
                    color='gray'  # Gray color for subtitle
                )
            )
        ]
    )
    fig.show()


### Plotar rebalanceamento diário vs. no alerta

Argumentos da função:
1. `inicio`: 'YYYY-MM-DD'
2. `fim`: 'YYYY-MM-DD'
3. `carteiras`: ***exc*** (obrigatório), ***hb*** e/ou *lc* (opcionais)

In [ ]:
def plot_comp_rebalanceamento(inicio, fim, carteira):


    df_ajustado = carteira.df_posicao_balanceada_alerta[carteira.nome]
    df_ajustado = df_ajustado[inicio:fim].pct_change().fillna(0)
    df_ajustado = (1+df_ajustado).cumprod()-1
    df_ajustado = df_ajustado.to_frame(name=carteira.nome + ' (Adj.)')

    df = carteira.calcular_retornos_acumulados(inicio,fim,ponderar=True)
    df = df[[carteira.nome]]

    data_frames = [df, df_ajustado]
    df = pd.concat(data_frames, axis=1)

    df = df * 100


    # Plota o gráfico usando Plotly
    fig = go.Figure()

    for col in df.columns:
        fig.add_trace(go.Scatter(x=df.index, y=df[col], mode='lines', name=col,
                                 hovertemplate='%{y:.2f}%',
                                 line_shape='spline',  # Makes the lines smoother
                                 line=dict(width=2)))  # Adjust line width


    titulo = 'Rebalanceamento diário vs. Rebalanceamento nos alertas'

    # Update layout to make it cleaner
    fig.update_layout(

        margin=dict(t=60),

        title={'text': titulo,
               'x': 0.055,  # Centering the title
               'y': 0.97,  # Position adjustment to leave space for subtitle
               'font': {'size': 18,  # Larger font size for title
                        'color': 'black',
                        'family': 'Georgia'}},  # Elegant serif font for title
        xaxis_title='',
        yaxis_title='Rentabilidade Acumulada (%)',
        legend_title="",
        hovermode='x unified',
        plot_bgcolor='white',  # White background
        xaxis=dict(
            showline=True,
            showgrid=True,
            gridcolor='lightgray',  # Light gray grid lines
            linecolor='white',  # Remove border
            linewidth=2,
            zerolinecolor='lightgray'
        ),
        yaxis=dict(
            showline=True,
            showgrid=True,
            gridcolor='lightgray',
            linecolor='white',
            linewidth=2,
            zerolinecolor='lightgray'
        ),
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=1.02,
            xanchor='right',
            x=1
        ),
        font=dict(
            family='Georgia',  # Update font family (optional)
            size=15,  # Adjust font size
            color='black'  # Font color
        ),
        annotations=[  # Adding the subtitle
            dict(
                text=f'Período: {inicio} à {fim}',  # Subtitle text
                xref='paper', yref='paper',  # Reference to paper (entire figure)
                x=0.00001, y=1.065,  # Position of subtitle (underneath title)
                showarrow=False,  # We don't need an arrow pointing to our subtitle
                font=dict(
                    family='Georgia',
                    size=13,
                    color='gray'  # Gray color for subtitle
                )
            )
        ]
    )
    fig.show()


### Plotar retorno dos ativos enquanto na carteira (não ponderado)

Argumentos da função:
1. `inicio`: 'YYYY-MM-DD'
2. `fim`: 'YYYY-MM-DD'
3. `carteira`: *exc*, *hb* ou *lc*

In [ ]:
def plot_ativos(inicio, fim, carteira):

    df = carteira.df_retornos[inicio:fim]

    df_aloc = carteira.df_aloc[inicio:fim]
    df_aloc = df_aloc.applymap(lambda x: float(x.strip('%')) / 100 if isinstance(x, str) and '%' in x else x)
    df_aloc = df_aloc.applymap(lambda x: 1 if x != 0 else 0)

    df = df.copy().mul(df_aloc)

    df.iloc[0] = 0

    colunas_excluir=df.loc[:, (df == 0).all(axis=0)].columns

    df = df.drop(columns=colunas_excluir)

    df = (1+df).cumprod() - 1

    last_row = df.iloc[-1].sort_values()
    last_row = last_row * 100

    last_row.to_csv('resultado_ativos.csv')

# Define as cores das barras com base nos valores
    colors = ['rgb(255, 99, 71)' if value < 0 else 'rgb(50, 205, 50)' for value in last_row.values]

    hover_texts = [f'Rentabilidade: {value:.2f}%' for index, value in zip(last_row.index, last_row.values)]

    # Plota o gráfico de barras
    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=last_row.index,
        y=last_row.values,
        marker_color=colors,
        marker_line_color='rgb(0,0,0,0)',
        marker_line_width=0.1,
        hoverinfo='y',
        hovertemplate='<b>%{x}</b>: %{y:.2f}%<extra></extra>',
        hoverlabel=dict(
            bgcolor='white',
            font_size=12,
            font_family='Georgia'
        ),
        showlegend=False
    ))


    # Ajustes no layout
    fig.update_layout(
        margin=dict(t=60),
        title={'text': f'{carteira.nome}: performance acumulada dos ativos',
               'x': 0.055,
               'y': 0.97,
               'font': {'size': 18, 'color': 'black', 'family': 'Georgia'}},
        xaxis_title='',
        yaxis_title='Rentabilidade Acumulada (%)',
        plot_bgcolor='white',
        xaxis=dict(
            showline=True,
            showgrid=False,
            linecolor='white',
            linewidth=2,
        ),
        yaxis=dict(
            showline=True,
            showgrid=True,
            gridcolor='lightgray',
            linecolor='white',
            linewidth=2,
            zerolinecolor='lightgray'
        ),
        font=dict(
            family='Georgia',
            size=15,
            color='black'
        ),
        annotations=[
            dict(
                text=f'Período: {inicio} à {fim}',
                xref='paper', yref='paper',
                x=0.0000005, y=1.065,
                showarrow=False,
                font=dict(family='Georgia', size=13, color='gray')
            )
        ]
    )
    fig.show()



### Plotar comparação entre carteira e ativos

Argumentos da função:
**negrito**
1. `inicio`: 'YYYY-MM-DD'
2. `fim`: 'YYYY-MM-DD'
3. `carteira`: *exc*, *hb* ou *lc*

In [ ]:
def plot_carteira_vs_ativos(inicio, fim, carteira):
    # Pegar o DataFrame com os retornos acumulados ponderados
    df = carteira.calcular_retornos_acumulados(inicio, fim, ponderar=False)
    df = df[carteira.calcular_retornos_acumulados(inicio, fim, ponderar=True).columns.intersection(df.columns)]

    # Pegar a última linha do DataFrame (última data) e ordenar
    last_row = df.iloc[-1].sort_values()
    last_row = last_row * 100

    last_row.to_csv(f'{carteira.nome}_resultado_{inicio}_{fim}.csv')

    colors = ['gray' if col != carteira.nome else 'rgb(187,13,49)' for col in last_row.index]

    hover_texts = [f'Rentabilidade: {value:.2f}%' for index, value in zip(last_row.index, last_row.values)]

    # Plota o gráfico de barras
    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=last_row.index,
        y=last_row.values,
        marker_color=colors,
        marker_line_color='rgb(0,0,0,0)',
        marker_line_width=0.1,
        hoverinfo='y',
        hovertemplate='<b>%{x}</b>: %{y:.2f}%<extra></extra>',
        hoverlabel=dict(
            bgcolor='white',
            font_size=12,
            font_family='Georgia'
        ),
        showlegend=False
    ))



    # Ajustes no layout
    fig.update_layout(
        margin=dict(t=60),
        title={'text': f'Performance Acumulada: {carteira.nome} vs. Ativos',
               'x': 0.055,
               'y': 0.97,
               'font': {'size': 18, 'color': 'black', 'family': 'Georgia'}},
        xaxis_title='',
        yaxis_title='Rentabilidade Acumulada (%)',
        plot_bgcolor='white',
        xaxis=dict(
            showline=True,
            showgrid=False,
            linecolor='white',
            linewidth=2,
        ),
        yaxis=dict(
            showline=True,
            showgrid=True,
            gridcolor='lightgray',
            linecolor='white',
            linewidth=2,
            zerolinecolor='lightgray'
        ),
        font=dict(
            family='Georgia',
            size=15,
            color='black'
        ),
        annotations=[
            dict(
                text=f'Período: {inicio} à {fim}',
                xref='paper', yref='paper',
                x=0.0000005, y=1.065,
                showarrow=False,
                font=dict(family='Georgia', size=13, color='gray')
            )
        ]
    )
    fig.show()



### Plotar decomposição de retorno acumulado

Argumentos da função:
1. `inicio`: 'YYYY-MM-DD'
2. `fim`: 'YYYY-MM-DD'
3. `carteira`: *exc*, *hb* ou *lc*
4. `segmentar`: True ou False (se True, agrega os retornos nas classes BTC, ETH e Alts)
5. `Acumular`: True ou False (se True, retorna valorizações acumuladas)

In [ ]:
def plot_decomposicao(inicio, fim, carteira, segmentar=True, acumular=False):

    if acumular:

      df = carteira.calcular_retornos_acumulados(inicio, fim, ponderar=True)
      df = df.drop(columns=[carteira.nome])


    else:

        df = carteira.df_retornos_ponderados[inicio:fim]
        df = df.loc[:, ~(df == 0).all(axis=0)]
        df = df.drop(columns=[carteira.nome])


    df = df.drop(df.index[0])
    df = df * 100




    # Se segmentar for True, agrupe todos os ativos exceto BTC e ETH em Altcoins
    if segmentar:
        # Verifique a existência de BTC e ETH
        btc_exists = 'BTC' in df.columns
        eth_exists = 'ETH' in df.columns

        # Crie a coluna Altcoins somando todos os ativos que não são BTC ou ETH
        altcoins_cols = [col for col in df.columns if col not in ['BTC', 'ETH']]
        df['Altcoins'] = df[altcoins_cols].sum(axis=1)

        # Exclua as colunas originais agora agrupadas em Altcoins
        df = df.drop(columns=altcoins_cols)

        # Reordene as colunas para que Altcoins apareça após BTC e ETH (se existirem)
        order = []
        if btc_exists:
            order.append('BTC')
        if eth_exists:
            order.append('ETH')
        order.append('Altcoins')
        df = df[order]

    if acumular:
        y_titulo = 'Retornos Ponderados e Acumulados (%)'
    else:
        y_titulo = 'Retornos Diários Ponderados (%)'


    titulo = f'Decomposição de Retorno (USD): {carteira.nome}'

    # Inicializa o gráfico
    fig = go.Figure()

    # Adicionar barras ao gráfico para cada coluna (ativo) no DataFrame
    for col in df.columns:
        fig.add_trace(go.Bar(
            x=df.index,
            y=df[col].values,
            name=col,
            hoverinfo='y',
            hovertemplate='<b>' + col + '</b>: %{y:.4f}%<extra></extra>',
            hoverlabel=dict(
                bgcolor='white',
                font_size=12,
                font_family='Georgia'
            ),
            showlegend=True,
            marker=dict(line=dict(width=0))
        ))

    # Ajustes no layout
    fig.update_layout(
        margin=dict(t=60),
        title={
            'text': titulo,
            'x': 0.055,
            'y': 0.97,
            'font': {'size': 18, 'color': 'black', 'family': 'Georgia'}
        },
        xaxis_title='',
        yaxis_title=y_titulo, # mudar
        plot_bgcolor='white',
        xaxis=dict(
            showline=True,
            showgrid=False,
            linecolor='white',
            linewidth=2,
        ),
        yaxis=dict(
            showline=True,
            showgrid=True,
            gridcolor='lightgray',
            linecolor='white',
            linewidth=2,
            zerolinecolor='lightgray'
        ),
        font=dict(
            family='Georgia',
            size=15,
            color='black'
        ),
        annotations=[
            dict(
                text=f'Período: {inicio} à {fim}',
                xref='paper',
                yref='paper',
                x=0.0000005,
                y=1.065,
                showarrow=False,
                font=dict(family='Georgia', size=13, color='gray')
            )
        ],
        barmode='relative'
    )

    # Mostra o gráfico
    fig.show()

### Plotar percentual de dias positivos/negativos

Argumentos da função:
1. `inicio`: 'YYYY-MM-DD'
2. `fim`: 'YYYY-MM-DD'
3. `carteira`: *exc*, *hb* ou *lc*
4. `segmentar`: True ou False (se True, agrega os retornos nas classes BTC, ETH e Alts)

In [ ]:
def plot_dias_pos_neg(inicio, fim, carteira, segmentar=False):
    # Pegar o DataFrame com as contribuições de retorno
    df = carteira.df_retornos_ponderados[inicio:fim]
    df = df.loc[:, ~(df == 0).all(axis=0)]
    df = df.drop(columns=[carteira.nome])
    df = df.drop(df.index[0])

    if segmentar:
        altcoins = [col for col in df.columns if col not in ['BTC', 'ETH']]
        df['Altcoins'] = df[altcoins].sum(axis=1)
        df.drop(columns=altcoins, inplace=True)

    pos_percents = {}
    neg_percents = {}
    fora_da_carteira_percents = {}

    for col in df.columns:
        pos_count = len(df[df[col] > 0])
        neg_count = len(df[df[col] < 0])
        fora_da_carteira_count = len(df[df[col] == 0])
        total_days = len(df)

        pos_percents[col] = round((pos_count / total_days), 4) * 100
        neg_percents[col] = round((neg_count / total_days), 4) * 100
        fora_da_carteira_percents[col] = round((fora_da_carteira_count / total_days), 4) * 100

    # Inicializa o gráfico
    fig = go.Figure()

    # Adicionar barras de porcentagens

    fig.add_trace(go.Bar(
        x=list(pos_percents.values()),
        y=list(pos_percents.keys()),
        orientation='h',
        name='Dias Positivos (%)',
        marker_color='green',
        hovertemplate='<b>%{y}</b>: %{x: .2f}%<extra></extra>'
    ))
    fig.add_trace(go.Bar(
        x=list(neg_percents.values()),
        y=list(neg_percents.keys()),
        orientation='h',
        name='Dias Negativos (%)',
        marker_color='red',
        hovertemplate='<b>%{y}</b>: %{x: .2f}%<extra></extra>'
    ))
    fig.add_trace(go.Bar(
        x=list(fora_da_carteira_percents.values()),
        y=list(fora_da_carteira_percents.keys()),
        orientation='h',
        name='Fora da Carteira (%)',
        marker_color='lightgray',
        hovertemplate='<b>%{y}</b>: %{x: .0f}%<extra></extra>'
    ))


    # Ajustes no layout
    fig.update_layout(
        margin=dict(t=60),
        title={
            'text': f'Percentual de Dias Positivos/Negativos: {carteira.nome}',
            'x': 0.055,
            'y': 0.97,
            'font': {'size': 18, 'color': 'black', 'family': 'Georgia'}
        },
        xaxis_title='',
        yaxis_title='',
        plot_bgcolor='white',
        xaxis=dict(
            showline=True,
            showgrid=True,
            linecolor='white',
            linewidth=2,

        ),
        yaxis=dict(
            showline=True,
            showgrid=False,
            gridcolor='lightgray',
            linecolor='white',
            linewidth=2,
            zerolinecolor='lightgray'
        ),
        font=dict(
            family='Georgia',
            size=15,
            color='black'
        ),
        annotations=[
            dict(
                text=f'Período: {inicio} à {fim}',
                xref='paper',
                yref='paper',
                x=0.0000005,
                y=1.065,
                showarrow=False,
                font=dict(family='Georgia', size=13, color='gray')
            )
        ],
        barmode='stack'
    )

    # Mostra o gráfico
    fig.show()


### Plotar histórico de alocação

Argumentos da função:
1. `inicio`: 'YYYY-MM-DD'
2. `fim`: 'YYYY-MM-DD'
3. `carteira`: *exc*, *hb* ou *lc*
4. `segmentar`: True ou False (se True, agrega os retornos nas classes BTC, ETH e Alts)
5. `real_aloc`: True ou False (se True, simula alocações sem rebalanceamento diário)

In [ ]:
def plot_alocacoes(inicio, fim, carteira, segmentar=False, real_aloc=False):
    # Pegar o DataFrame com as alocações

    if real_aloc:
      df = carteira.df_aloc_balanceada_alerta[inicio:fim]
    else:
      df = carteira.df_aloc[inicio:fim]
      df = df.applymap(lambda x: float(x.strip('%')) / 100 if isinstance(x, str) and '%' in x else x) # por cause que o df de alocações vem em percentual, do sheets

    df = df.loc[:, ~(df == 0).all(axis=0)]
    df = df * 100

    # Se segmentar for True, faça as alterações necessárias
    if segmentar:
        # Agrupe 'USDT', 'BUSD' e 'TUSD' em 'Caixa' se estiverem no DataFrame
        caixa_cols = ['USDT', 'BUSD', 'TUSD']
        caixa_presentes = [col for col in caixa_cols if col in df.columns]
        if caixa_presentes:
            df['Caixa'] = df[caixa_presentes].sum(axis=1)
            df = df.drop(columns=caixa_presentes)

        # Agrupe todos os ativos exceto 'BTC', 'ETH' e 'Caixa' em 'Altcoins'
        altcoins_cols = [col for col in df.columns if col not in ['BTC', 'ETH', 'Caixa']]
        if altcoins_cols:
            df['Altcoins'] = df[altcoins_cols].sum(axis=1)
            df = df.drop(columns=altcoins_cols)

    # Inicializa o gráfico
    fig = go.Figure()

    # Adicione barras ao gráfico para cada coluna (ativo) no DataFrame
    for col in df.columns:
        fig.add_trace(go.Bar(
            x=df.index,
            y=df[col].values,
            name=col,
            hovertemplate='<b>' + col + '</b>: %{y:.0f}%<extra></extra>',
            marker=dict(line=dict(width=0))
        ))

    # Ajustes no layout
    fig.update_layout(
        margin=dict(t=60),
        title={
            'text': f'Alocação de Ativos ao Longo do Tempo: {carteira.nome}',
            'x': 0.055,
            'y': 0.97,
            'font': {'size': 18, 'color': 'black', 'family': 'Georgia'}
        },
        xaxis_title='',
        yaxis_title='Alocação (%)',
        plot_bgcolor='white',
        bargap=0,
        xaxis=dict(
            showline=True,
            showgrid=False,
            linecolor='white',
            linewidth=0,
            tickangle=-45, # rotacionar rótulos do eixo X
            nticks=20 # limita a quantidade de ticks no eixo X
        ),
        yaxis=dict(
            showline=False,
            showgrid=False,
            gridcolor='lightgray',
            linecolor='white',
            linewidth=2,
            zerolinecolor='lightgray'
        ),
        font=dict(
            family='Georgia',
            size=15,
            color='black'
        ),
        annotations=[
            dict(
                text=f'Período: {inicio} à {fim}',
                xref='paper',
                yref='paper',
                x=0.0000005,
                y=1.065,
                showarrow=False,
                font=dict(family='Georgia', size=13, color='gray')
            )
        ],
        barmode='stack'
    )

    # Mostra o gráfico
    fig.show()


### Plotar Value At Risk e Value of Oportunity

*-- Value at Risk (VaR):*

**Definição**: O VaR é uma medida estatística que estima o risco máximo de perda financeira que um portfólio de investimentos pode sofrer em um determinado período de tempo, dado um nível específico de confiança.

*-- Value of Opportunity:*

**Definição**: Embora não seja um termo padrão como o VaR, "Value of Opportunity" pode ser entendido como o inverso do VaR. Em vez de focar nas perdas potenciais, ele foca nos ganhos potenciais.

<br></br>
Argumentos da função:
1. `carteira`: *exc*, *hb* ou *lc*
2. `janela_amostral`: quantidade de dias passados que será levado em conta na estatística
3. `nivel_confianca`: nível de confiança (padrão é 95%)
4. `periodo`: periodo para estimativa futura em dias (precisa ser <= `janela_amostral`)
5. `filtrar_atuais`: True ou False (se True, inclue somente ativos atualmente na carteira; se False, todos os ativos que já fizeram parte da carteira)

In [ ]:
def plot_var_voo(carteira, janela_amostral=90, nivel_confianca=95, periodo=7, filtrar_atuais=False):
    # Cortar a planilha de alocações para a janela de amostral
    df_aloc_cortada = carteira.df_aloc.tail(janela_amostral)
    df_aloc_cortada = df_aloc_cortada.applymap(lambda x: float(x.strip('%')) / 100 if isinstance(x, str) and '%' in x else x)

    colunas_para_excluir = ["USDT", "BUSD", "TUSD"]

    if filtrar_atuais:
        # Obter o último registro de alocação e verificar quais ativos têm alocação diferente de zero
        ativos_atualmente = df_aloc_cortada.tail(1)
        ativos_atualmente = ativos_atualmente.loc[:, (ativos_atualmente != 0).any(axis=0)]
        # Filtrar esses ativos no DataFrame de retornos e excluir os indesejados
        df_retornos_filtrados = carteira.df_retornos[ativos_atualmente.columns].drop(columns=colunas_para_excluir, errors='ignore')
    else:
        # Utilizar todos os ativos da carteira.df_retornos excluindo as colunas indesejadas
        df_retornos_filtrados = carteira.df_retornos.drop(columns=colunas_para_excluir, errors='ignore')



    # Calcular VaR e VoO para cada ativo
    var_values = []
    voo_values = []
    ativos = []

    ativos_diferenca = []

    for ativo in df_retornos_filtrados.columns:
        try:
            var = carteira.calcular_var_historico(ativo, nivel_confianca, janela_amostral, periodo)
            voo = carteira.calcular_voo_historico(ativo, nivel_confianca, janela_amostral, periodo)
            ativos_diferenca.append((ativo, var, voo))
        except ValueError:
            # Ignora ativos que não estão presentes na carteira
            continue

    # Ordenar pela diferença entre VoO e VaR (ordem decrescente de diferencial)
    ativos_diferenca.sort(key=lambda x: x[2] + x[1], reverse=True)

    # Desempacotar os ativos e valores após a ordenação
    ativos, var_values, voo_values = zip(*ativos_diferenca)
    # Plota o gráfico de barras
    fig = go.Figure()

    # Adicionar barras para o VaR
    fig.add_trace(go.Bar(
        x=ativos,
        y=var_values,
        name='VaR',
        marker_color='rgb(255, 99, 71)',
        hoverinfo='y',
        hovertemplate='<b>' + 'VaR' + '</b>: %{y:.2f}%<extra></extra>',
    ))

    # Adicionar barras para o VoO
    fig.add_trace(go.Bar(
        x=ativos,
        y=voo_values,
        name='VoO',
        marker_color='rgb(50, 205, 50)',
        hoverinfo='y',
        hovertemplate='<b>' + 'VoO' + '</b>: %{y:.2f}%<extra></extra>',
    ))

    # Encontrar o valor mínimo e máximo para definir o range do eixo y
    min_value = min(min(var_values), min(voo_values)) * 1.1
    max_value = max(max(var_values), max(voo_values)) * 1.1

    # Ajustes no layout, incluindo anotação com parâmetros utilizados
    fig.update_layout(
        barmode='relative',
        title={'text': f'Risco x Oportunidade (VaR x VoO): {carteira.nome}',
               'x': 0.5,
               'y': 0.95,
               'font': {'size': 18, 'color': 'black', 'family': 'Georgia'}},
        xaxis_title='',
        yaxis_title='Percentual (%)',
        plot_bgcolor='white',
        xaxis=dict(
            showline=True,
            showgrid=False,
            linecolor='black',
            linewidth=2,
        ),
        yaxis=dict(
            showline=True,
            showgrid=True,
            gridcolor='lightgray',
            linecolor='black',
            linewidth=2,
            zeroline=True,
            zerolinewidth=2,
            zerolinecolor='black',
            range=[min_value, max_value]
        ),
        font=dict(
            family='Georgia',
            size=15,
            color='black'
        ),
        annotations=[
            dict(
                x=0.01,
                y=1.2,
                showarrow=False,
                text=f"<br>Janela Amostral: {janela_amostral} dias<br>"
                     f"Nível de Confiança: {nivel_confianca}%<br>"
                     f"Período: {periodo} dias",
                xref="paper",
                yref="paper",
                align="left",
                bordercolor="black",
                borderwidth=1,
                borderpad=4,
                bgcolor="white",
                font=dict(
                    family="Georgia",
                    size=12,
                    color="black"
                )
            )
        ]
    )

    # Exibe o gráfico
    fig.show()


### Plotar matriz de correlação

Argumentos da função:
1. `carteira`: *exc*, *hb* ou *lc*
2. `janela_amostral`: quantidade de dias passados que será levado em conta na estatística
3. `filtrar_atuais`: True ou False (se True, inclue somente ativos atualmente na carteira; se False, todos os ativos que já fizeram parte da carteira)
4. `setorizar`: True ou False (se True, agrega os ativos em narrativas com base na planilha de categorias)

In [ ]:
def plot_correlation_matrix(carteira, janela_amostral=30, filtrar_atuais=False, setorizar=False):

    carteira.df_aloc=carteira.df_aloc.applymap(lambda x: float(x.strip('%')) / 100 if isinstance(x, str) and '%' in x else x)

    if filtrar_atuais:
        ativos_atualmente = carteira.df_aloc.tail(1)
        ativos_atualmente = ativos_atualmente.loc[:, (ativos_atualmente != 0).any(axis=0)]
        df_retornos_filtrados = carteira.df_retornos[ativos_atualmente.columns]

    else:
        df_retornos_filtrados = carteira.df_retornos

    colunas_para_excluir = ["USDT", "BUSD", "TUSD"]
    df_retornos_filtrados = df_retornos_filtrados.drop(columns=colunas_para_excluir, errors='ignore')


    if setorizar:

        if not hasattr(carteira, 'df_cats'):
            raise ValueError("A carteira não possui o atributo 'df_cats' necessário para setorizar.")


        df_cats = carteira.df_cats
        retornos_por_categoria = {}

        for categoria in df_cats['categoria'].unique():
            ativos_na_categoria = df_cats[df_cats['categoria'] == categoria]['ativo']
            ativos_na_categoria = [ativo for ativo in ativos_na_categoria if ativo in df_retornos_filtrados.columns]

            # Calcular o retorno médio para a categoria
            if ativos_na_categoria:
                retornos_por_categoria[categoria] = df_retornos_filtrados[ativos_na_categoria].median(axis=1)


        df_retornos_filtrados = pd.DataFrame(retornos_por_categoria)



    df_retornos_filtrados=df_retornos_filtrados.copy().tail(janela_amostral)



    corr_matrix = df_retornos_filtrados.corr()


    # Criar o gráfico de calor para a matriz de correlação
    fig = go.Figure(data=go.Heatmap(
        z=corr_matrix.values,
        x=corr_matrix.columns,
        y=corr_matrix.index,
        colorscale='Viridis',
        colorbar=dict(title='Correlação'),
    ))

    # Ajustes no layout
    fig.update_layout(
        title={'text': f'Matriz de Correlação de {janela_amostral} dias - {carteira.nome}',
               'x': 0.5,
               'y': 0.95,
               'xanchor': 'center',
               'yanchor': 'top'},
        xaxis_title='',
        yaxis_title='',
        xaxis=dict(tickangle=-45),
        yaxis=dict(tickangle=45),
        font=dict(
            family='Georgia',
            size=12,
            color='black'
        ),
        hovermode='closest',
    )

    # Exibe o gráfico
    fig.show()



### Plotar alocações por ativo ao longo do tempo

Essa função funciona plota um Gantt Chart que mostra entradas e saídas de ativos ao longo do tempo em uma carteira.

Argumentos da função:
1. `carteira`: *exc*, *hb* ou *lc*


In [ ]:
import plotly.express as px
import pandas as pd

def posicoes(carteira):

    df_aloc = carteira.df_aloc.applymap(lambda x: float(x.strip('%')) / 100 if isinstance(x, str) and '%' in x else x)

    tasks = []
    for ativo in df_aloc.columns:
        # Encontrar os períodos em que o ativo está na carteira
        in_portfolio = df_aloc[ativo] != 0
        starts = df_aloc.index[in_portfolio & (~in_portfolio.shift(1, fill_value=False))].tolist()
        ends = df_aloc.index[in_portfolio & (~in_portfolio.shift(-1, fill_value=False))].tolist()

        # Se o ativo está na carteira desde o início, adicione a primeira data como início
        if in_portfolio.iloc[0]:
            starts.insert(0, df_aloc.index[0])

        # Se o ativo está na carteira no final, adicione a última data como fim
        if in_portfolio.iloc[-1]:
            ends.append(df_aloc.index[-1])

        # Adicionar os períodos ao conjunto de tarefas
        tasks += [{'Ativo': ativo, 'Start': s, 'Finish': e} for s, e in zip(starts, ends)]

    df_tasks = pd.DataFrame(tasks)

    # Criar o gráfico de Gantt
    fig = px.timeline(df_tasks, x_start="Start", x_end="Finish", y="Ativo", labels={'Ativo':'Ativos'})


    altura_por_ativo = 30
    altura_total = altura_por_ativo * len(df_aloc.columns)

    # Ajustar o layout para corresponder à estética desejada e definir a altura dinâmica
    fig.update_layout({
        'title': 'Posições dos Ativos ao Longo do Tempo',
        'xaxis_title': 'Data',
        'yaxis_title': '',
        'showlegend': False,
        'plot_bgcolor': 'white',
        'font': dict(family="Georgia", size=15, color="black"),
        'xaxis': {
            'showline': True,
            'showgrid': False,
            'linecolor': 'black',
            'linewidth': 2
        },
        'yaxis': {
            'showgrid': True,
            'gridcolor': 'lightgray',
            'showline': True,
            'linecolor': 'black',
            'linewidth': 2
        },
        'height': altura_total  # Define a altura dinamicamente
    })

    # Exibir o gráfico
    fig.show()


## Setup inicial

Abaixo são definidas os ativos em cada carteira e importadas as planilhas de alocação.

#### Para adicionar um ativo novo:

1. Procurar pelo ativo no CoinGecko (i.e. https://www.coingecko.com/pt/moedas/dogwifhat)
2. Na página do ativo, localizar o id de API (i.e. 'dogwifcoin')
3. Adicionar o ativo na lista da carteira respectiva
4. Adicionar uma nova coluna com o **ticker** do ativo na planilha da carteira respectiva, e preencher as alocações com 0 até o dia da primeira alocação.


[Link das planilhas aqui!](https://docs.google.com/spreadsheets/d/1156S3pm9vQ2JP5bE0yvpj_Y8FE55ecovHS_JvIys5dc)

#### Para retirar um ativo:

1. Basta zerar a posição do ativo na planilha de alocações.

In [ ]:
carteira_EXC = ['aave', 'arbitrum', 'arweave','audius','aurory','avalanche-2','axie-infinity','balancer','band-protocol','binancecoin','bitcoin-cash','cardano',
                'eos','litecoin','polkadot','ripple','bitcoin','curve-dao-token','tether','true-usd','chainlink','compound-governance-token','cosmos','crypto-com-chain',
                'dash','decentraland', 'dodo', 'dydx','ethereum', 'ethereum-classic', 'fantom', 'flow', 'gmx', 'stellar', 'havven', 'helium', 'internet-computer',
                'kyber-network-crystal', 'lido-dao','lisk', 'maker', 'matic-network', 'monero', 'polymath', 'near', 'omisego', 'optimism', 'pancakeswap-token', 'pax-gold',
                'pendle', 'star-atlas-dao', 'ronin', 'rari-governance-token', 'the-sandbox', 'secret', 'serum', 'smartcash', 'solana', 'spell-token', 'sushi', 'swarm',
                'terra-luna-2', 'tezos', 'tribe-2', 'uniswap', 'wibx', 'wrapped-nxm', 'yearn-finance', 'zcash', 'frax-share', 'celestia', 'ronin', 'thorchain', 'immutable-x',
                'akash-network', 'render-token']

carteira_HB = ["ethereum", "tether", "maker", "havven", "aave", "uniswap", "dydx",
              "cosmos", "secret", "the-sandbox", "helium", "matic-network",
              "near", "decentraland",
              "curve-dao-token", "gmx", "optimism",
              "gala", "flow", "lido-dao", "frax-share", "numeraire",
              "gains-network", "injective-protocol", "arbitrum", "pendle", "radiant-capital",
              "chainlink", "solana", "avalanche-2", "kujira", "echelon-prime", "akash-network", "render-token", "multibit", "beam-2"]

carteira_LC = ["arweave", "badger-dao", "my-neighbor-alice", "perpetual-protocol",
              "alpha-finance", "yield-guild-games", "genopets", "acala","rainbow-token-2",
              "guild-of-guardians", "aurory", "illuvium", "conic-finance", "vela-token",
              "radiant-capital", "botto", "pendle", "nunet", "kryptonite", "prisma-governance-token",
              "genesysgo-shadow", "neon", "mintlayer"]


worksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1156S3pm9vQ2JP5bE0yvpj_Y8FE55ecovHS_JvIys5dc')

def sheet_to_dataframe(worksheet):

    df = pd.DataFrame(worksheet)
    df.columns = df.iloc[0]
    df = df[1:]

    if 'Data' in df.columns:
      df.set_index('Data', inplace=True)
      df.index = pd.to_datetime(df.index, format='%Y-%m-%d')

    return df


exc_aloc = sheet_to_dataframe(worksheet.worksheet('EXC').get_all_values())
hb_aloc = sheet_to_dataframe(worksheet.worksheet('HB').get_all_values())
lc_aloc = sheet_to_dataframe(worksheet.worksheet('LC').get_all_values())
cats = sheet_to_dataframe(worksheet.worksheet('CATS').get_all_values())



## Instanciando as carteiras

Abaixo, criamos as intâncias de cada carteira de acordo com os requerimentos da classe `PortfolioCrypto()`.


Ao rodar esse código, o script vai primeiramente atualizar os preços de todos os ativos, em cada pasta individual. É possível implementar um cron job para automatizar esse processo.

In [ ]:
%%time

exc = PortfolioCrypto(carteira='EXC',
                      ativos=carteira_EXC,
                      aloc=exc_aloc,
                      categorias=cats)


hb = PortfolioCrypto(carteira='High Beta',
                     ativos=carteira_HB,
                     aloc=hb_aloc,
                     categorias=cats)


lc = PortfolioCrypto(carteira='Low Caps',
                     ativos=carteira_LC,
                     aloc=lc_aloc,
                     categorias=cats)

Processando 1 de 76: aave
Processando 2 de 76: arbitrum
Processando 3 de 76: arweave
Processando 4 de 76: audius
Processando 5 de 76: aurory
Processando 6 de 76: avalanche-2
Processando 7 de 76: axie-infinity
Processando 8 de 76: balancer
Processando 9 de 76: band-protocol
Processando 10 de 76: binancecoin
Processando 11 de 76: bitcoin-cash
Processando 12 de 76: cardano
Processando 13 de 76: eos
Processando 14 de 76: litecoin
Processando 15 de 76: polkadot
API cooldown (1m)...
Processando 16 de 76: ripple
Processando 17 de 76: bitcoin
Processando 18 de 76: curve-dao-token
Processando 19 de 76: tether
Processando 20 de 76: true-usd
Processando 21 de 76: chainlink
Processando 22 de 76: compound-governance-token
Processando 23 de 76: cosmos
Processando 24 de 76: crypto-com-chain
Processando 25 de 76: dash
Processando 26 de 76: decentraland
Processando 27 de 76: dodo
Processando 28 de 76: dydx
Processando 29 de 76: ethereum
Processando 30 de 76: ethereum-classic
API cooldown (1m)...
Proces

# Diagnóstico de carteiras


## Exponential Coins

In [ ]:
exc.posicoes_abertas()

In [ ]:
inicio = '2023-12-31'
fim = '2024-01-10'
betas=hb.rolling_beta(inicio=inicio,
                      fim=fim,
                      janela=1)



In [ ]:
#betas

In [ ]:


import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(betas.index, betas['High Beta'], label='Beta Semanal')
historical_mean = betas['High Beta'].mean()
print(historical_mean)
#plt.axhline(y=historical_mean, color='red', linestyle='--', label='Média Histórica')
plt.xlabel('')
plt.ylabel('Beta')
plt.title(f'Beta semanal da carteira High Beta')
plt.legend()
plt.show()



In [ ]:
hb.posicoes_abertas()

In [ ]:
lc.posicoes_abertas()

In [ ]:
inicio = '2024-01-09'
fim = '2024-01-10'

#fim = (datetime.now() - timedelta(days=1)).date()

plot_carteiras(inicio, fim, exc, brl=True)
plot_decomposicao(inicio, fim, exc, segmentar=True, acumular=True)
plot_carteira_vs_ativos(inicio, fim, exc)
plot_ativos(inicio,fim,exc)
plot_dias_pos_neg(inicio, fim, exc, segmentar=True)

plot_alocacoes(inicio,fim,exc, segmentar=True)





## High beta

In [ ]:
#inicio = '2023-11-30'
#fim = '2023-12-18'


plot_decomposicao(inicio, fim, hb, segmentar=True, acumular=True)
plot_carteira_vs_ativos(inicio, fim, hb)
plot_ativos(inicio,fim,hb)
plot_dias_pos_neg(inicio, fim, hb, segmentar=True)





## Low Caps

In [ ]:
#inicio = '2023-11-30'
#fim = '2023-12-18'

plot_decomposicao(inicio, fim, lc, segmentar=False, acumular=True)
plot_carteira_vs_ativos(inicio, fim, lc)
plot_ativos(inicio,fim,lc)
plot_dias_pos_neg(inicio, fim, lc, segmentar=False)

In [ ]:
# gráfico para saber quanto um token (ou uma cesta de tokens) retornou desde a entrada na carteira

# Medidas estatísticas de risco: VaR, hitting ratio, Sharpes, Sortinos, Calmars, StDev, etc

# Selling effectiviness


# Estudo de rebalanceamento somente durante alertas
# Estudo de rebalanceamento mensal
# planilha de posições


# Rebalanceamentos até os alertas

#exc.df_precos.to_csv("precos_exc.csv")




In [ ]:
plot_alocacoes(inicio, fim, exc)

In [ ]:
plot_alocacoes(inicio,fim,exc,segmentar=True)
plot_alocacoes(inicio,fim,hb,segmentar=True)
plot_alocacoes(inicio,fim,lc,segmentar=False)